##  *Develop an agent that can achieve a specific goal using a sub-goal in order to accelarate learnign in a  maze 10X10 based on RL algorithm (Q-learning).*

In [14]:
# Import the necessary libraries to create the maze and give our agent the ability to randomly making choices

import numpy as np
import random
import torch

####  Setting up the environment WITH SUB-GOAL

In [15]:
# Implemetation of the maze 10X10 as indicated in the assignment. 

# 1 represents the walls (unpassable)
# 0 represents an open path (passable)
# S marks the starting point of the agent
# SG marks the sub-goal the agent should reach before continuing
# End marks the end goal

maze = np.array([[1,1,1,1,1,1,1,1,1,1],
                 [1,'S',1,0,0,0,1,0,0,1],
                 [1,0,1,0,1,0,1,0,1,1],
                 [1,0,0,0,1,0,0,0,0,1],
                 [1,1,1,0,1,1,1,1,0,1],
                 [1,0,1,'SG',0,0,0,1,0,1],
                 [1,0,0,0,1,1,0,0,0,1],
                 [1,1,1,0,1,0,1,1,0,1],
                 [1,0,0,0,0,0,1,'End',0,1],
                 [1,1,1,1,1,1,1,1,1,1]], dtype=object)



# Implementatio of the Q-Table
# Considering the maze above, and the Q-learning algorithm, we need to initiate
# the Q-table. The Q-table include all possible states-actions pairs and their corresponing Q-values.

q_table = np.zeros((10,10,4))   # each states have 4 possible actions (up, down, left, right)
actions = ['up', 'down', 'left', 'right']

# Implementation of the agent's starting position in the maze, the sub-goal and the end-goal

start_position = (1,1)
sub_goal = (5,3)
end_goal = (8,7)


# Implementation of the reward system

reward_sub_goal = 10
reward_end_goal = 100
penalty = -1    

# Implementation of the agent's interactions with the environment, the way the agent moves in the maze
# and the way penalties and rewards are given to the agent.

def moves_subgoal(state, actions):
    row, col = state

    if actions == 'up':
        next_state = (row - 1, col)
    elif actions == 'down':
        next_state = (row + 1, col)
    elif actions == 'left':
        next_state = (row, col - 1)
    elif actions == 'right':
        next_state = (row, col + 1)
    else:
        next_state = state

# Check if the next state is a wall or out of the maze
    if 0 <= next_state[0] < 10 and 0 <= next_state[1] < 10:
        if maze[next_state] != 1:
            if next_state == sub_goal:
                return next_state, reward_sub_goal, False   #Sub-goal reached 
            elif next_state == end_goal:
                return next_state, reward_end_goal, True    #End-goal reached
            else:
                return next_state, penalty, False           #No goal reached, keep searching
        else:
            return state, penalty, False                    #Hit a wall, stay in the same state
    else:
        return state, penalty, False                        #Out of the maze, stay in the same state



# Implementation of the starting funtion, in order to restart the agent's position in the maze
# at the end of each episode.

def restarting():
    return start_position


# Implementation of the selection of the agent's action.
# The agent will randomly choose the actions to take in the Q-table, based on the epsilon-greedy policy.
# The action with the highest Q-value will be chosen 

def select_action(state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return np.random.randint(len(actions))
    else:
        return np.argmax(q_table[state[0], state[1]])

####  Setting up the environment WITHOUT SUB-GOAL

In [16]:
# Implemetation of the maze 10X10 as indicated in the assignment. 

# 1 represents the walls (unpassable)
# 0 represents an open path (passable)
# S marks the starting point of the agent
# End marks the end goal

maze = np.array([[1,1,1,1,1,1,1,1,1,1],
                 [1,'S',1,0,0,0,1,0,0,1],
                 [1,0,1,0,1,0,1,0,1,1],
                 [1,0,0,0,1,0,0,0,0,1],
                 [1,1,1,0,1,1,1,1,0,1],
                 [1,0,1,0,0,0,0,1,0,1],
                 [1,0,0,0,1,1,0,0,0,1],
                 [1,1,1,0,1,0,1,1,0,1],
                 [1,0,0,0,0,0,1,'End',0,1],
                 [1,1,1,1,1,1,1,1,1,1]], dtype=object)



# Implementatio of the Q-Table
# Considering the maze above, and the Q-learning algorithm, we need to initiate
# the Q-table. The Q-table include all possible states-actions pairs and their corresponing Q-values.

q_table = np.zeros((10,10,4))   # each states have 4 possible actions (up, down, left, right)
actions = ['up', 'down', 'left', 'right']

# Implementation of the agent's starting position in the maze, the sub-goal and the end-goal

start_position = (1,1)
end_goal = (8,7)


# Implementation of the reward system

reward_end_goal = 100
penalty = -1    

# Implementation of the agent's interactions with the environment, the way the agent moves in the maze
# and the way penalties and rewards are given to the agent.

def moves_NO_subgoal(state, actions):
    row, col = state

    if actions == 'up':
        next_state = (row - 1, col)
    elif actions == 'down':
        next_state = (row + 1, col)
    elif actions == 'left':
        next_state = (row, col - 1)
    elif actions == 'right':
        next_state = (row, col + 1)
    else:
        next_state = state

# Check if the next state is a wall or out of the maze
    if 0 <= next_state[0] < 10 and 0 <= next_state[1] < 10:
        if maze[next_state] != 1:
            if next_state == end_goal:
                return next_state, reward_end_goal, True    #End-goal reached
            else:
                return next_state, penalty, False           #No goal reached, keep searching
        else:
            return state, penalty, False                    #Hit a wall, stay in the same state
    else:
        return state, penalty, False                        #Out of the maze, stay in the same state



# Implementation of the starting funtion, in order to restart the agent's position in the maze
# at the end of each episode.

def restarting():
    return start_position


# Implementation of the selection of the agent's action.
# The agent will randomly choose the actions to take in the Q-table, based on the epsilon-greedy policy.
# The action with the highest Q-value will be chosen 

def select_action(state, epsilon):
    if random.uniform(0, 1) < epsilon:
        return np.random.randint(len(actions))
    else:
        return np.argmax(q_table[state[0], state[1]])

#### Hyperparameters

In [22]:
alpha = 0.1  # Learning rate
gamma = 0.1  # Discount factor
epsilon = 1.0  # Initial exploration rate. 
epsilon_variation = 0.99  # This parameter will decrease the epsilon value over time, in order to make the agent more greedy
min_epsilon = 0.01  # Minimum exploration rate reached by the agent
episodes = 10000 # Number of episodes the agent will run
steps = 100  # Number of steps the agent will take in each episode

#### Implementation of the Q-learning algorithm

In [18]:
# Implementation of the Q-learning algorithm based on the mathematical formula
# Q(s,a) = Q(s,a) + alpha * (reward + gamma * max(Q(s',a')) - Q(s,a))

def q_value(state, action_index, reward, next_state):
    
    max_q_value = np.max(q_table[next_state[0], next_state[1]])
    actual_q_value = q_table[state[0], state[1], action_index]
    new_q_value = actual_q_value + alpha * (reward + gamma * max_q_value - actual_q_value)

    q_table[state[0], state[1], action_index] = new_q_value

#### Implementation of the training process

In [23]:
for episode in range(episodes):
    state = restarting()
    done = False
    total_reward = 0

    #if episode < 5:  # Print debug information for the first few episodes
    #    print(f"\nStarting Episode {episode + 1}")

    for step in range(steps):
        action_index = select_action(state, epsilon)
        action = actions[action_index]

        # Debug: Print selected action and state
    #    if episode < 5:
    #        print(f"Step {step + 1}: State: {state}, Action: {action}")

        next_state, reward, done = moves_subgoal(state, action)

        # Debug: Print next state, reward, and done flag
    #    if episode < 5:
    #        print(f"Next State: {next_state}, Reward: {reward}, Done: {done}")

        # Update Q-value
        max_future_q = np.max(q_table[next_state[0], next_state[1]])
        current_q = q_table[state[0], state[1], action_index]
        q_table[state[0], state[1], action_index] = current_q + alpha * (reward + gamma * max_future_q - current_q)

        state = next_state
        total_reward += reward

        if done:
    #        if episode < 5:
    #            print(f"Episode {episode + 1} completed after {step + 1} steps with total reward {total_reward}")
           break

    # Decay epsilon
    epsilon = max(min_epsilon, epsilon * epsilon_variation)

    # Print progress every 100 episodes
    if (episode + 1) % 100 == 0:
        print(f'Episode {episode + 1}/{episodes}, Total reward: {total_reward}')


Episode 100/10000, Total reward: 48
Episode 200/10000, Total reward: 373
Episode 300/10000, Total reward: 395
Episode 400/10000, Total reward: 428
Episode 500/10000, Total reward: 428
Episode 600/10000, Total reward: 428
Episode 700/10000, Total reward: 406
Episode 800/10000, Total reward: 406
Episode 900/10000, Total reward: 428
Episode 1000/10000, Total reward: 428
Episode 1100/10000, Total reward: 428
Episode 1200/10000, Total reward: 428
Episode 1300/10000, Total reward: 428
Episode 1400/10000, Total reward: 428
Episode 1500/10000, Total reward: 417
Episode 1600/10000, Total reward: 428
Episode 1700/10000, Total reward: 417
Episode 1800/10000, Total reward: 428
Episode 1900/10000, Total reward: 417
Episode 2000/10000, Total reward: 428
Episode 2100/10000, Total reward: 428
Episode 2200/10000, Total reward: 428
Episode 2300/10000, Total reward: 417
Episode 2400/10000, Total reward: 406
Episode 2500/10000, Total reward: 428
Episode 2600/10000, Total reward: 428
Episode 2700/10000, To

#### extras for checking the performance of the agent

In [20]:
import cProfile

# Define your evaluation function
def evaluate_policy():
    sub_goal_reach_count = 0
    total_rewards = torch.zeros(100)  # Assuming you're not using CUDA for now
    steps_taken = torch.zeros(100)

    for episode in range(100):  # Set the number of evaluation episodes
        state = torch.tensor(restarting())
        done = False
        episode_reward = 0
        episode_steps = 0

        while not done and episode_steps < 1000:  # Limit steps per episode
            action_index = select_action((state[0].item(), state[1].item()), 0)  # Always choose the best action
            action = actions[action_index]
            next_state, reward, done = moves((state[0].item(), state[1].item()), action)
            next_state = torch.tensor(next_state)

            if (state[0].item(), state[1].item()) == sub_goal:
                sub_goal_reach_count += 1

            state = next_state
            episode_reward += reward
            episode_steps += 1

        total_rewards[episode] = episode_reward
        steps_taken[episode] = episode_steps

    print(f"Average Reward per Episode: {torch.mean(total_rewards):.2f}")
    print(f"Average Steps per Episode: {torch.mean(steps_taken):.2f}")
    print(f"Sub-goal Reach Count: {sub_goal_reach_count}")

# Run the profiler on the evaluation function
cProfile.run('evaluate_policy()')


Average Reward per Episode: 4478.00
Average Steps per Episode: 1000.00
Sub-goal Reach Count: 49700
         1600277 function calls in 4.550 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.162    0.000    0.162    0.000 1712913629.py:45(moves)
      100    0.000    0.000    0.000    0.000 3126056008.py:73(restarting)
   100000    0.269    0.000    0.752    0.000 3126056008.py:81(select_action)
        1    2.429    2.429    4.550    4.550 745427005.py:4(evaluate_policy)
        1    0.000    0.000    4.550    4.550 <string>:1(<module>)
        2    0.001    0.000    0.001    0.000 _tensor.py:929(__format__)
   100000    0.018    0.000    0.018    0.000 fromnumeric.py:1136(_argmax_dispatcher)
   100000    0.136    0.000    0.373    0.000 fromnumeric.py:1140(argmax)
   100000    0.085    0.000    0.237    0.000 fromnumeric.py:53(_wrapfunc)
        1    0.000    0.000    0.000    0.000 iostream.py:203(schedule)
 